In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd


! pip install datasets transformers sacrebleu

model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"

from datasets import load_dataset, load_metric

raw_datasets = load_dataset("wmt16", "ru-en")
metric = load_metric("sacrebleu")

! pip install sentencepiece

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

if "mbart" in model_checkpoint:
    tokenizer.src_lang = "ru-RU"
    tokenizer.tgt_lang = "en-XX"

if model_checkpoint in [ "t5-base"]:
    prefix = "translate Russian to English: "
else:
    prefix = ""

max_input_length = 128
max_target_length = 128
source_lang = "ru"
target_lang = "en"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy = "epoch",
    learning_rate=10e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

  Using cached https://files.pythonhosted.org/packages/fd/1a/41c644c963249fd7f3836d926afa1e3f1cc234a1c40d80c5f03ad8f6f1b2/transformers-4.8.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/57/0c7ca4e31a126189dab99c19951910bd081dea5bbd25f24b77107750eae7/sacrebleu-1.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/89/a6/3814b7107e0788040870e8825eebf214d72166adf656ba7d4bf14759a06a/portalocker-2.0.0-py2.py3-none-any.whl
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ru-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.



     |████████████████████████████████| 1.2MB 29.1MB/s 


Using amp fp16 backend
The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 1516162
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 94761


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,2.175800,1.504680,29.015500,26.467400


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1299: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to test-translation/checkpoint-500
Configuration saved in test-translation/checkpoint-500/config.json
Model weights saved in test-translation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-translation/checkpoint-1000
Configuration saved in test-translation/checkpoint-1000/config.json
Model weights saved in test-translation/checkpoint-1000/pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=94761, training_loss=2.42046151580007, metrics={'train_runtime': 16412.0065, 'train_samples_per_second': 92.381, 'train_steps_per_second': 5.774, 'total_flos': 1.05593737700352e+17, 'train_loss': 2.42046151580007, 'epoch': 1.0})

In [ ]:
model.save_pretrained("/content/drive/My Drive/News article summarizer/Ru_En")
tokenizer.save_pretrained("/content/drive/My Drive/News article summarizer/Ru_En")

Configuration saved in /content/drive/My Drive/News article summarizer/Ru_En/config.json
Model weights saved in /content/drive/My Drive/News article summarizer/Ru_En/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/News article summarizer/Ru_En/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/News article summarizer/Ru_En/special_tokens_map.json


('/content/drive/My Drive/News article summarizer/Ru_En/tokenizer_config.json',
 '/content/drive/My Drive/News article summarizer/Ru_En/special_tokens_map.json',
 PosixPath('/content/drive/My Drive/News article summarizer/Ru_En/source_spm'),
 PosixPath('/content/drive/My Drive/News article summarizer/Ru_En/target_spm'),
 PosixPath('/content/drive/My Drive/News article summarizer/Ru_En/vocab'),
 PosixPath('/content/drive/My Drive/News article summarizer/Ru_En/tokenizer_config_file'),
 '/content/drive/My Drive/News article summarizer/Ru_En/added_tokens.json')